<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag：半结构化数据的并行列表

Dask-bag 擅长处理可以表示为任意输入序列的数据。我们将其称为“杂乱”数据，因为它可能包含复杂的嵌套结构、缺失的字段、数据类型的混合等。 *函数式*编程风格非常适合标准Python迭代，例如可以在`itertools`模块。

当第一次消耗大量原始数据时，在数据处理管道的开始阶段经常会遇到凌乱的数据。初始数据集可能是JSON、CSV、XML或任何其他不强制执行严格结构和数据类型的格式。
出于这个原因，最初的数据按摩和处理通常是用Python的 `list`、`dict`和 `set`来完成的。

这些核心数据结构针对通用存储和处理进行了优化。使用迭代器/生成器表达式或诸如`itertools` 或 [`toolz`](https://toolz.readthedocs.io/en/latest/) 之类的库添加流式计算，让我们可以在小空间内处理大量数据。如果我们将其与并行处理相结合，那么我们可以处理大量数据。

Dask.bag是一个高级Dask集合，用于自动化这种形式的常见工作负载。简而言之，

    dask.bag = map, filter, toolz + 并行执行
    
**相关文档**

* [Bag 文档](https://docs.dask.org/en/latest/bag.html)
* [Bag 截屏](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag 样例](https://examples.dask.org/bag.html)

## 创建数据

In [ ]:
%run prep.py -d accounts

## 设置

同样，我们将使用分布式调度器。 调度器将在[以后](05_distributed.ipynb)更详细地解释。

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

## 创建

您可以从Python序列、从文件、从S3上的数据等创建`Bag`。
我们演示了使用 `.take()` 来显示数据元素。 （执行`.take(1)` 会生成一个包含一个元素的元组）

注意数据是分块的，每个块有很多项。 在第一个示例中，两个分区各包含五个元素，在接下来的两个示例中，每个文件被划分为一个或多个字节块。

In [ ]:
# 每个元素都是一个整数
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

In [ ]:
# 每个元素都是一个文本文件，其中每一行都是一个 JSON 对象
# 注意压缩是自动处理的
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

In [ ]:
# 编辑sources.py以配置源位置
import sources
sources.bag_url

In [ ]:
# 需要`s3fs`库
# 每个分区都是一个远程 CSV 文本文件 
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

## 操作

`Bag` 对象包含了Python 标准库、`toolz` 或 `pyspark` 等项目中找到的标准功能 API，包括 `map`、`filter`、`groupby` 等。

对`Bag`对象的操作会创建新的`Bag`。 调用`.compute()` 方法来触发执行，正如我们在`Delayed` 对象中看到的那样。

In [ ]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

In [ ]:
# 阻塞形式：等待完成（在这种情况下非常快）
c.compute()

### 示例：Accounts JSON 数据

我们在您的数据目录中创建了一个gzip压缩的JSON 数据的假数据集。 这类似于我们稍后将看到的 `DataFrame` 示例中使用的示例，不同之处在于它将每个单独的 `id` 的所有条目捆绑到单个记录中。 这类似于您可能从文档存储数据库或 Web API 收集的数据。

每一行都是一个 JSON 编码的字典，带有以下键

* id：客户的唯一标识符
* name：客户姓名
* transaction-：`transaction-id`、`amount` 对列表，该文件中客户的每笔交易各一笔。

In [ ]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

我们的数据以文本行的形式从文件中出来。请注意，文件解压缩是自动进行的。通过将`json.loads`函数映射到bag中，我们可以使这些数据看起来更合理。

In [ ]:
import json
js = lines.map(json.loads)
# take: 检查前几个元素
js.take(3)

### 基本查询

一旦我们将 JSON 数据解析为适当的 Python 对象（`dict`、`list`等），我们就可以通过创建小型Python函数来运行我们的数据，以执行更有趣的查询。

In [ ]:
# filter: 仅保留序列的某些元素
js.filter(lambda record: record['name'] == 'Alice').take(5)

In [ ]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

In [ ]:
# pluck: 从字典中选择一个字段， element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

In [ ]:
# 所有 Alice 条目的平均交易数
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

### 使用`flatten` 去除嵌套

在下面的示例中，我们看到使用 `.flatten()` 来平铺结果。 我们计算所有 Alice 的所有交易的平均金额。

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

In [ ]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

### Groupby 和 Foldby

通常我们想通过一些函数或键对数据进行分组。 我们可以使用 `.groupby` 方法来做到这一点，该方法很简单，但会强制对数据进行完整的混洗（代价很高），或者使用更难使用但速度更快的 `.foldby` 方法，它将groupby和归约结合起来进行流式处理

* `groupby`：混洗数据，使所有具有相同键的项目都在相同的键值对中
* `foldby`：遍历数据，每个键累积一个结果

*注意：完整的 groupby 特别糟糕。 在实际工作负载中，如果可能，您最好使用 `foldby` 或切换到 `DataFrame`。*

### `groupby`

Groupby收集集合中的项，以便将某些函数下具有相同值的所有项收集到一个键值对中。

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

In [ ]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

### `foldby`

Foldby初看起来可能很奇怪。它类似于其他库中的以下函数：

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)
*  
使用 `foldby` 时，您提供

1. 对元素进行分组的关键函数
2. 一个二元运算符，比如你会传递给 `reduce` 的，您可以使用该运算符对每个组执行归约
3. 组合二元运算符，可以组合对数据集不同部分的两次`reduce`”`调用的结果。

你的归约必须是关联的。 它会在数据集的每个分区中并行发生。 然后所有这些中间结果将由`combine`二元运算符组合。

In [ ]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

### account数据的示例

我们找到同名的人数

In [ ]:
%%time
# 警告，这将花费一定时间...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

In [ ]:
%%time
# 这个速度相对较快，并产生相同的结果。
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

### 练习：计算每个名字的总金额

我们想要groupby (或 foldby) `name` 键, 然后将每个名字的所有金额相加。

步骤

1. 创建一个小函数，给定一个字典

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    计算amount的总和, 如： `3`
    
2. 稍微更改上面“foldby”示例的二元运算符，以便二元运算符不计算条目的数量，而是累加数量的总和。

In [ ]:
# 将你的代码写在这里。。。

## DataFrames

出于与 Pandas 通常比纯 Python 更快的相同原因，`dask.dataframe` 可以比 `dask.bag` 更快。 稍后我们将更多地使用 DataFrames，但从 Bag 的角度来看，它通常是数据摄取“混乱”部分的终点 —— 一旦数据可以制成dataframe，然后进行复杂的拆分-应用-合并(`split-apply-combine`) 逻辑将变得更加直接和高效。

您可以使用 `to_dataframe` 方法将具有简单元组或平面字典结构的包转换为 `dask.dataframe`。

In [ ]:
df1 = js.to_dataframe()
df1.head()

这现在看起来像一个定义良好的 DataFrame，我们可以有效地对其应用类似 Pandas 的计算。

使用 Dask DataFrame，我们事先计算同名人数需要多长时间？ 事实证明，`dask.dataframe.groupby()` 比`dask.bag.groupby()` 高出一个数量级以上； 但在这个案例中，它仍然无法比得上`dask.bag.foldby()`。

In [ ]:
%time df1.groupby('name').id.count().compute().head()

### 非规范化 

这种 DataFrame 格式不是最佳的，因为 `transactions` 列充满了嵌套数据，所以 Pandas 必须恢复到 `object`数据类型，这在 Pandas 中很慢。 理想情况下，我们希望仅在我们将数据展平后才转换为数据帧，以便每条记录都是单独的 `int`、`string`、`float` 等。

In [ ]:
def denormalize(record):
    # 为每个人返回一个列表，每笔交易一个项目
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

In [ ]:
df = transactions.to_dataframe()
df.head()

In [ ]:
%%time
# 每个名字的交易数量
# 注意这里的时间包括数据加载和摄取
df.groupby('name')['transaction-id'].count().compute()

## 限制

Bags 提供了非常通用的计算（任何 Python 函数）。这种通用性是有代价的。 Bags 具有以下已知限制：

1. Bag 操作往往比数组/数据帧计算慢，就像 Python 比 NumPy/Pandas 慢一样
2. ``Bag.groupby`` 很慢。 如果可能，您应该尝试使用`Bag.foldby`。
     使用``Bag.foldby`` 需要更多的思考。 更好的是，考虑创建一个规范化的数据框。

## 了解更多

* [Bag 文档](https://docs.dask.org/en/latest/bag.html)
* [Bag 截屏视频](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag 示例](https://examples.dask.org/bag.html)

## 关闭

In [ ]:
client.shutdown()